In [16]:
import numpy as np
import parmed as pmd

# MosDef packages
import foyer
from foyer import Forcefield
import freud
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import hoomd
import hoomd.md
import hoomd.group

In [2]:
# User inputs

In [3]:
def compound(smiles_string):
    
    comp_mb = mb.load(smiles_string, smiles = True) # creates an mbuild compound instance
    #comp_pmd = comp_mb.to_parmed() # convert the mbuild compound to a parmed compound
    #comp_mass = np.sum([atom.mass for atom in comp_pmd.atoms]) # Mass of the compound
    #comp_mb.mass = comp_mass # Update mb compound's mass

    #GAFF = foyer.forcefields.load_GAFF()
    #comp_typed = GAFF.apply(comp_mb, assert_dihedral_params=False)
    #for atom_pmd, atom_mb in zip(comp_typed, comp_mb):
    #    atom_mb.name = "{}".format(atom_pmd.type)
    return comp_mb

def build_box(compound, num_compounds):
    
    #TO DO: Add, and use density parameter to determine box edge lengths
    
    GAFF = foyer.forcefields.load_GAFF()
    box = mb.Box([10,10,10])
    system_box = mb.packing.fill_box(compound, num_compounds, box=box)
    system_box_pmd = system_box.to_parmed(box=box, infer_residues = True) # line 2347 of mbuild
    # What is happening here that breaks the ff information?
    system_box_pmd = GAFF.apply(system_box_pmd, assert_dihedral_params=False)
    
        # Why can't we start with an mbuild box of untyped compounds, then handle the typing here?
        # What is different about applying the ff to the box, vs creating a box with already typed molecules?
        # Where/How does Hoomd know which force field to use?
    
    return system_box_pmd

def sim(system, steps, kT, tau):
    log_quantities = ['temperature', 'pressure', 'volume', 'potential_energy', 'kinetic_energy']
    create_hoomd_simulation(system, r_cut=1.2, auto_scale=True)
    _all = hoomd.group.all()
    hoomd.md.integrate.mode_standard(dt=0.0001)
    integrator = hoomd.md.integrate.nvt(group=_all, kT=kT, tau=tau)
    integrator.randomize_velocities(seed=42)
    hoomd.analyze.log("trajectory.log", log_quantities, period=500, overwrite=True)
    hoomd.dump.gsd("start.gsd", period=None, group=_all, overwrite=True)
    hoomd.dump.gsd("traj.gsd", period=1e3, group=_all, phase=0, overwrite=True)
    hoomd.run(steps)
    hoomd.dump.gsd("out.gsd", period=None, group=_all, overwrite=True);
    
def rdf():
    pass

In [4]:
p3ht = compound('c1sccc1CCCCCC')
system = build_box(p3ht, 10)
sim(system, steps=3e5, kT=1, tau=1)

/home/chris/miniconda3/envs/mosdef-smiles/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/home/chris/cme/mbuild/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule c1sccc1CCCCCC	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/chris/cme/forks/foyer/foyer/forcefield.py:433: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/home/chris/cme/forks/foyer/foyer/forcefield.py:445: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/home/chris/cme/forks/foyer/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", Validatio

HOOMD-blue 2.9.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 02/04/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Lui, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 270 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 3 exclusions             : 30
notice(2): Particles with 4 exclusions             : 130
notice(2): Particles with 6 exclusions             : 30
notice(2): Particles with 7 exclusions             : 20


/home/chris/cme/forks/foyer/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 670, Parameterized dihedrals: 630. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)
/home/chris/cme/forks/foyer/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 280, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)


Time 00:00:20 | Step 104345 / 300000 | TPS 5361.44 | ETA 00:00:36
Time 00:00:30 | Step 157192 / 300000 | TPS 5284.64 | ETA 00:00:27
Time 00:00:40 | Step 211625 / 300000 | TPS 5443.3 | ETA 00:00:16
Time 00:00:50 | Step 265594 / 300000 | TPS 5396.87 | ETA 00:00:06
Time 00:00:56 | Step 300000 / 300000 | TPS 5432.1 | ETA 00:00:00
Average TPS: 5325.28
---------
-- Neighborlist stats:
0 normal updates / 3000 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 20 / n_neigh_avg: 6.52593
shortest rebuild period: 100
-- Cell list stats:
Dimension: 17, 17, 17
n_min    : 0 / n_max: 18 / n_avg: 0.0549562
** run complete **


In [19]:
import gsd
import gsd.hoomd
import gsd.fl
gsd_file = gsd.fl.GSDFile('traj.gsd', "rb", "MyApp", "hoomd", [1,0])
trajectory = gsd.hoomd.HOOMDTrajectory(gsd_file)
x, y, z = trajectory[-1].configuration.box[:3]
r_max = x // 2
dr = 0.25
freud_box = freud.box.Box(x, y, z)
rdf = freud.density.RDF(r_max, dr)

num_frames = 3

def single_frame_rdf(frame, types, rdf, box):
    
    if types[0] == 'all':
        positions = frame.particles.position
    else:
        positions = []
        #for atom_type in types:
        for idx, type_id in enumerate(frame.particles.typeid):
            if frame.particles.types[type_id] in types:
                positions.append(frame.particles.position[idx])
    print('# OF PARTICLES = {}'.format(len(positions)))
    rdf.accumulate(box=box, ref_points = positions)
    return rdf